In [ ]:
!pip install fastai transformers torch huggingFace

In [ ]:
import fastai
import torch
import transformers
import huggingface_hub
from fastai.text.all import *
import random
import shutil
from pathlib import Path

In [ ]:
import random
import shutil
from pathlib import Path
import os
from fastai.text.all import *

# Function to sample a subset of text files
def sample_text_files(path: Path, sample_fraction: float = 0.1, seed: int = 42) -> Path:
    """
    Sample a subset of text files from the dataset.

    Parameters:
    - path: Path to the dataset directory.
    - sample_fraction: Fraction of the dataset to sample.
    - seed: Random seed for reproducibility.

    Returns:
    - Path to a temporary directory containing the sampled files.
    """
    random.seed(seed)

    # Create temporary directory for sampled data
    sample_path = path / 'sampled'
    sample_path.mkdir(parents=True, exist_ok=True)

    # Collect all text file paths
    all_files = list(path.rglob('*.txt'))

    # Sample a fraction of files
    sampled_files = random.sample(all_files, int(len(all_files) * sample_fraction))

    # Copy sampled files to the new directory
    for file in sampled_files:
        dest = sample_path / file.relative_to(path)
        dest.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(file, dest)

    return sample_path

# Load the IMDB dataset
path = untar_data(URLs.IMDB)

# Sample a subset of the dataset
sampled_path = sample_text_files(path, sample_fraction=0.1, seed=42)

# Create DataLoaders from the sampled dataset
dls = TextDataLoaders.from_folder(
    sampled_path,
    valid='test',          # Ensure this is correct, or use valid_pct if not structured
    text_col='text',      # Use this only if applicable
    label_col='label',    # Use this only if applicable
    valid_pct=0.2,        # Fraction of data to use for validation
    seed=42,
    bs=32,                # Batch size
    tok_tfm=Tokenizer.from_df # Add a tokenizer
)

# Print DataLoaders to confirm
print(dls)

FileNotFoundError: [Errno 2] No such file or directory: '/root/.fastai/data/imdb/sampled_tok/sampled/train/pos/2913_7.txt'

In [ ]:
dls.show_batch()

## Task 1: Fine tuning and transfer learning with fastai

**Citation and all rights reserved to:**
*Applied Natural Language Processing in the Enterprise Teaching Machines to Read, Write & Understand by Ankur A. Patel & Ajay Uppili Arasanipalai.*


I followed the instructions on pages (35-55) of the textbook to achieve my result. I also made some necessary or desirable modifications, **which I consider to be for public use without any rights.**

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn.fine_tune(4, 1e-2)

In [ ]:
learn.fine_tune(4, 1e-2)

In [ ]:
learn.show_results()

In [ ]:
learn.predict("That movie was wicked cool!")
('pos', tensor(1), tensor([0.0092, 0.9908]))

In [ ]:
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, valid_pct=0.1)
dls_lm.show_batch(max_n=5)

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()],
                                path=path, wd=0.1).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.save('1epoch')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

In [ ]:
learn.save_encoder('finetuned')

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
        for _ in range(N_SENTENCES)]

print("\n".join(preds))

In [ ]:
dls_clas = TextDataLoaders.from_folder(
    untar_data(URLs.IMDB), valid='test',
    text_vocab=dls_lm.vocab)

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn = learn.load_encoder('finetuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pretrained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "With great power comes great " indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

In [ ]:
print(tokens_tensor)

In [ ]:
# Load pretrained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# Predict all tokens
with torch.no_grad():
  outputs = model(tokens_tensor)
  predictions = outputs[0]

# Get the predicted next subword
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print(predicted_text)

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
model.eval()

In [ ]:
with torch.no_grad():
  outputs = model(tokens_tensor)
  predictions = outputs[0]

In [ ]:
len(outputs)
outputs[0].shape
torch.Size([1, 5, 50257])

In [ ]:
text = "With great power comes great "

In [ ]:
with torch.no_grad():
  outputs = model(tokens_tensor)
  predictions = outputs[0]

In [ ]:
predicted_index = torch.argmax(predictions[0, -1, :]).item()

In [ ]:
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print(predicted_text)

## Task 2: Build RNN model for text classification


**Citation and all rights reserved to:**
*Applied Natural Language Processing in the Enterprise Teaching Machines to Read, Write & Understand by Ankur A. Patel & Ajay Uppili Arasanipalai.*


I followed the instructions on pages (131-148) of the textbook to achieve my result. I also made some necessary or desirable modifications, **which I consider to be for public use without any rights.**

In [ ]:
model = torch.nn.RNN(300, 512)

In [ ]:
for word in words:
  state = f(word, state)